## Изследване на клъстеризация с K-Means чрез R и принципите на Tidy данни.

### [**Тест преди лекцията**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

В този урок ще научите как да създавате клъстери, използвайки пакета Tidymodels и други пакети от екосистемата на R (ще ги наричаме приятели 🧑‍🤝‍🧑), както и набора от данни за нигерийска музика, който импортирахте по-рано. Ще разгледаме основите на K-Means за клъстеризация. Имайте предвид, че както научихте в предишния урок, има много начини за работа с клъстери и методът, който използвате, зависи от вашите данни. Ще опитаме K-Means, тъй като това е най-често използваната техника за клъстеризация. Да започваме!

Термини, които ще научите:

-   Оценка на силуета

-   Метод на лакътя

-   Инерция

-   Вариация

### **Въведение**

[K-Means клъстеризация](https://wikipedia.org/wiki/K-means_clustering) е метод, произхождащ от областта на обработката на сигнали. Използва се за разделяне и групиране на данни в `k клъстери` въз основа на сходства в техните характеристики.

Клъстерите могат да бъдат визуализирани като [диаграми на Вороной](https://wikipedia.org/wiki/Voronoi_diagram), които включват точка (или 'семе') и съответната й област.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Инфографика от Джен Лупър</figcaption>

Стъпките на K-Means клъстеризацията са следните:

1.  Ученият по данни започва с определяне на желания брой клъстери, които да бъдат създадени.

2.  След това алгоритъмът случайно избира K наблюдения от набора от данни, които да служат като начални центрове за клъстерите (т.е. центроиди).

3.  След това всяко от останалите наблюдения се присвоява на най-близкия центроид.

4.  След това се изчислява новата средна стойност на всеки клъстер и центроидът се премества към тази средна стойност.

5.  Сега, когато центровете са преизчислени, всяко наблюдение отново се проверява, за да се види дали може да е по-близо до друг клъстер. Всички обекти се преразпределят отново, използвайки актуализираните средни стойности на клъстерите. Стъпките за преразпределение на клъстерите и актуализация на центроидите се повтарят итеративно, докато разпределението на клъстерите спре да се променя (т.е. когато се постигне конвергенция). Обикновено алгоритъмът приключва, когато всяка нова итерация води до незначително движение на центроидите и клъстерите стават статични.

<div>

> Имайте предвид, че поради случайността при избора на началните k наблюдения, използвани като стартови центроиди, можем да получим леко различни резултати всеки път, когато приложим процедурата. Поради тази причина повечето алгоритми използват няколко *случайни старта* и избират итерацията с най-нисък WCSS. Затова силно се препоръчва винаги да изпълнявате K-Means с няколко стойности на *nstart*, за да избегнете *нежелан локален оптимум.*

</div>

Тази кратка анимация, използваща [илюстрациите](https://github.com/allisonhorst/stats-illustrations) на Алисън Хорст, обяснява процеса на клъстеризация:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Илюстрация от @allison_horst</figcaption>

Един основен въпрос, който възниква при клъстеризацията, е следният: как да разберете на колко клъстери да разделите данните си? Един недостатък на използването на K-Means е фактът, че трябва да установите `k`, тоест броя на `центроидите`. За щастие, `методът на лакътя` помага да се оцени добра начална стойност за `k`. Ще го опитате след малко.

### 

**Предварителни изисквания**

Ще продължим точно оттам, където спряхме в [предишния урок](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), където анализирахме набора от данни, направихме множество визуализации и филтрирахме набора от данни до интересуващите ни наблюдения. Уверете се, че сте го разгледали!

Ще ни трябват някои пакети, за да завършим този модул. Можете да ги инсталирате със следната команда: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Алтернативно, скриптът по-долу проверява дали имате необходимите пакети за завършване на този модул и ги инсталира за вас, ако някои липсват.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Нека запретнем ръкави!

## 1. Танц с данните: Стеснете избора до трите най-популярни музикални жанра

Това е обобщение на това, което направихме в предишния урок. Нека анализираме и обработим малко данни!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Това мина добре!

## 2. Още изследване на данните.

Колко чисти са тези данни? Нека проверим за отклонения с помощта на box plot диаграми. Ще се съсредоточим върху числови колони с по-малко отклонения (въпреки че бихте могли да изчистите отклоненията). Box plot диаграмите могат да покажат диапазона на данните и ще помогнат да се изберат кои колони да се използват. Забележете, че box plot диаграмите не показват вариация, която е важен елемент за добри данни, подходящи за клъстеризация. Моля, вижте [тази дискусия](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) за допълнителна информация.

[Box plot диаграмите](https://en.wikipedia.org/wiki/Box_plot) се използват за графично представяне на разпределението на `числови` данни, така че нека започнем с *избор* на всички числови колони заедно с популярните музикални жанрове.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Вижте как помощникът за селекция `where` прави това лесно 💁? Разгледайте други подобни функции [тук](https://tidyselect.r-lib.org/).

Тъй като ще създаваме кутии за разпределение (boxplot) за всяка числова характеристика и искаме да избегнем използването на цикли, нека преформатираме данните си в *по-дълъг* формат, който ще ни позволи да се възползваме от `facets` - подграфики, всяка от които показва един подмножество от данните.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Много по-дълго! Сега е време за малко `ggplots`! И така, кой `geom` ще използваме?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Лесно-gg!

Сега можем да видим, че тези данни са малко шумни: като наблюдаваме всяка колона чрез boxplot, можем да забележим аутлайъри. Бихте могли да прегледате набора от данни и да премахнете тези аутлайъри, но това би направило данните доста минимални.

Засега нека изберем кои колони ще използваме за нашето упражнение по клъстеризация. Нека изберем числовите колони с подобни диапазони. Можем да кодираме `artist_top_genre` като числова стойност, но засега ще я пропуснем.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Изчисляване на k-means клъстериране в R

Можем да изчислим k-means в R с вградената функция `kmeans`, вижте `help("kmeans()")`. Функцията `kmeans()` приема дата фрейм с всички числови колони като основен аргумент.

Първата стъпка при използването на k-means клъстериране е да се определи броят на клъстерите (k), които ще бъдат генерирани в крайното решение. Знаем, че има 3 музикални жанра, които сме извлекли от набора от данни, така че нека опитаме с 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Обектът kmeans съдържа няколко части информация, които са добре обяснени в `help("kmeans()")`. Засега нека се съсредоточим върху няколко от тях. Виждаме, че данните са групирани в 3 клъстера с размери 65, 110, 111. Резултатът също така съдържа центровете на клъстерите (средните стойности) за трите групи през петте променливи.

Клъстеризиращият вектор представлява класификацията на клъстера за всяко наблюдение. Нека използваме функцията `augment`, за да добавим класификацията на клъстера към оригиналния набор от данни.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Перфектно, току-що разделихме нашия набор от данни на 3 групи. И така, колко добра е нашата кластеризация 🤷? Нека разгледаме `Silhouette score`.

### **Silhouette score**

[Silhouette анализът](https://en.wikipedia.org/wiki/Silhouette_(clustering)) може да се използва за изучаване на разстоянието на разделяне между получените клъстери. Тази оценка варира от -1 до 1, като ако оценката е близо до 1, клъстерът е плътен и добре отделен от другите клъстери. Стойност близо до 0 представлява припокриващи се клъстери с проби, които са много близо до границата на решение на съседните клъстери. [източник](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Методът за средна стойност на силуета изчислява средния силует на наблюденията за различни стойности на *k*. Висока средна стойност на силуета показва добра кластеризация.

Функцията `silhouette` в пакета за клъстери се използва за изчисляване на средната ширина на силуета.

> Силуетът може да бъде изчислен с всяка [метрика за разстояние](https://en.wikipedia.org/wiki/Distance "Distance"), като например [Евклидово разстояние](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") или [Манхатънско разстояние](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), които обсъдихме в [предишния урок](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Нашият резултат е **0.549**, което е точно по средата. Това показва, че нашите данни не са особено подходящи за този тип клъстеризация. Нека видим дали можем да потвърдим това предположение визуално. Пакетът [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) предоставя функции (`fviz_cluster()`) за визуализиране на клъстеризацията.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Покриването на клъстерите показва, че нашите данни не са особено подходящи за този тип клъстеризация, но нека продължим.

## 4. Определяне на оптималния брой клъстери

Основен въпрос, който често възниква при клъстеризация с K-Means, е следният - без известни етикети на класовете, как да разберем на колко клъстера да разделим данните си?

Един от начините да разберем това е като използваме извадка от данните, за да `създадем серия от модели за клъстеризация` с увеличаващ се брой клъстери (например от 1 до 10) и оценим метрики за клъстеризация като **Silhouette score.**

Нека определим оптималния брой клъстери, като изчислим алгоритъма за клъстеризация за различни стойности на *k* и оценим **Сумата на квадратите в рамките на клъстера** (WCSS). Общата сума на квадратите в рамките на клъстера (WCSS) измерва компактността на клъстеризацията и искаме тя да бъде възможно най-малка, като по-ниски стойности означават, че точките от данните са по-близо една до друга.

Нека изследваме ефекта от различните избори на `k`, от 1 до 10, върху тази клъстеризация.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Сега, когато имаме общата сума на квадратите в рамките на клъстера (tot.withinss) за всеки алгоритъм за клъстеризация с център *k*, използваме [метода на лакътя](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), за да намерим оптималния брой клъстери. Методът се състои в изчертаване на WCSS като функция от броя на клъстерите и избиране на [лакътя на кривата](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Лакът на кривата") като броя на клъстерите, които да се използват.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Графиката показва значително намаление на WCSS (следователно по-голяма *стегнатост*), когато броят на клъстерите се увеличава от един на два, и допълнително забележимо намаление от два на три клъстера. След това намалението е по-слабо изразено, което води до `лакът` 💪 на графиката около три клъстера. Това е добър показател, че има две до три сравнително добре разделени клъстера от точки данни.

Сега можем да продължим и да извлечем модела за клъстериране, където `k = 3`:

> `pull()`: използва се за извличане на една колона
>
> `pluck()`: използва се за индексиране на структури от данни като списъци


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Чудесно! Нека да визуализираме получените клъстери. Искате ли малко интерактивност с `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Може би бихме очаквали, че всеки клъстер (представен с различни цветове) ще има различни жанрове (представени с различни форми).

Нека разгледаме точността на модела.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Точността на този модел не е лоша, но не е и отлична. Възможно е данните да не са подходящи за K-Means Clustering. Тези данни са твърде небалансирани, слабо корелирани и има прекалено голяма вариация между стойностите на колоните, за да се формират добри клъстери. Всъщност, клъстерите, които се формират, вероятно са силно повлияни или изкривени от трите жанрови категории, които дефинирахме по-горе.

Въпреки това, процесът беше доста поучителен!

В документацията на Scikit-learn можете да видите, че модел като този, с клъстери, които не са добре разграничени, има проблем с "вариацията":

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Инфографика от Scikit-learn</figcaption>



## **Вариация**

Вариацията се определя като "средната стойност на квадратите на разликите от средната стойност" [източник](https://www.mathsisfun.com/data/standard-deviation.html). В контекста на този проблем с клъстерирането, това се отнася до данни, при които стойностите в набора от данни се отклоняват твърде много от средната стойност.

✅ Това е чудесен момент да помислите за всички начини, по които можете да коригирате този проблем. Да промените данните малко повече? Да използвате различни колони? Да използвате различен алгоритъм? Подсказка: Опитайте [скалиране на данните](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/), за да ги нормализирате и да тествате други колони.

> Опитайте този '[калкулатор за вариация](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', за да разберете концепцията малко по-добре.

------------------------------------------------------------------------

## **🚀Предизвикателство**

Отделете време с този notebook, за да промените параметрите. Можете ли да подобрите точността на модела, като почистите данните повече (например премахнете аутлайъри)? Можете да използвате тегла, за да дадете повече тежест на определени проби от данни. Какво друго можете да направите, за да създадете по-добри клъстери?

Подсказка: Опитайте да скалирате данните си. В notebook има коментиран код, който добавя стандартно скалиране, за да направи колоните с данни по-сходни по отношение на диапазона. Ще забележите, че докато оценката на силуета намалява, "пречупването" в графиката на лакътя се изглажда. Това е така, защото оставянето на данните без скалиране позволява данни с по-малка вариация да имат по-голяма тежест. Прочетете повече за този проблем [тук](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Тест след лекцията**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Преглед и самостоятелно обучение**

-   Разгледайте симулатор за K-Means [като този](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Можете да използвате този инструмент, за да визуализирате примерни точки от данни и да определите техните центроиди. Можете да редактирате случайността на данните, броя на клъстерите и броя на центроидите. Помага ли това да получите представа как данните могат да бъдат групирани?

-   Също така, разгледайте [този материал за K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) от Станфорд.

Искате ли да приложите новопридобитите си умения за клъстериране към набори от данни, които са подходящи за K-Means клъстериране? Вижте:

-   [Обучение и оценка на модели за клъстериране](https://rpubs.com/eR_ic/clustering) с помощта на Tidymodels и приятели

-   [Анализ на клъстери с K-Means](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [Клъстериране с K-Means и принципи на tidy данни](https://www.tidymodels.org/learn/statistics/k-means/)

## **Задание**

[Опитайте различни методи за клъстериране](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## БЛАГОДАРНОСТИ НА:

[Jen Looper](https://www.twitter.com/jenlooper) за създаването на оригиналната Python версия на този модул ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) за създаването на невероятните илюстрации, които правят R по-приветлив и ангажиращ. Намерете повече илюстрации в нейния [галерия](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Приятно учене,

[Eric](https://twitter.com/ericntay), Златен студентски посланик на Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Илюстрация от @allison_horst</figcaption>



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален превод от човек. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
